In [10]:
import pandas as pd
import numpy as np

import os
import glob

In [11]:
# Log file from psychopy.
path_search=os.path.join('..',
                    'data', 
                    'logfiles_task01', 
                    'sub-17_BrainplaybackQuadrants_*_run-1.csv')

log_fn = glob.glob(path_search)[0]
#print(path_search)
#print(log_fn)

In [12]:
#create a new lists by parsing the string using "," delimiter

head_tail = os.path.split(log_fn)
print(head_tail[1])

sub_id=head_tail[1].split("_")

print("Participant: ",sub_id[0])

run_id=sub_id[-1].split(".")
print("Run id: ",run_id[0])

sub-17_BrainplaybackQuadrants_2023-04-18_16h57.33.000_run-1.csv
Participant:  sub-17
Run id:  run-1


In [13]:
# run data
sub_id=sub_id[0]
run_id=run_id[0]

# default values
sess_id='01'
task_id='01permusicparticipant'


In [14]:
# Create pandas dataframe to read data into data structure.
df_log=pd.read_csv(log_fn)


In [15]:
# df_log.columns

In [16]:
# Select subset of columns
df_log_M=df_log.loc[:,["MusicsSelect", "musics","Baseline1.started", "Noise.started","Baseline2.started", "MusicExcerpt.started", "MusicExcerpt.stopped"]]



In [17]:
#df_log_M

## Get Quadrants selected by participant 

In [664]:
def getQ(row):  
    t = row['music.id'].split('/')
    return (f'{t[3]}')

def categorise(row):  
    if row['mouse.x'] > 0 and row['mouse.y'] > 0:
        return 'Q1'
    elif row['mouse.x'] < 0 and row['mouse.y'] > 0:
        return 'Q2'
    elif row['mouse.x'] < 0  and row['mouse.y'] < 0:
        return 'Q3'
    return 'Q4'

In [665]:
full_list = os.listdir(os.path.join('..',
                    'music_characterization_stim', 
                    'data'))


# list to store files
csv_file = []

# participant idx
p = sub_id.split('-')[1]

# run idx
r = run_id.split('-')[1]
r = str(r).zfill(2)

print("Participant (idx), run (idx): ",p, r)



# identify csv file for each run
tok_filename = f'{p}_{r}'

# Iterate directory
for file in full_list:
    # check only text files
    if file.startswith(tok_filename) and file.endswith('.csv'):
        csv_file.append(file)
        
df_Q = pd.read_csv(os.path.join('..',
                    'music_characterization_stim', 
                    'data', 
                    csv_file[0]))

# Last row NaN.
df_Q = df_Q[:24]
    
# Select rows of interest.
df_Q=df_Q[['music.id','mouse.x','mouse.y']]

# Extract position from mouse columns.
df_Q['mouse.x'] = df_Q['mouse.x'].str[1:-1].astype(float)
df_Q['mouse.y'] = df_Q['mouse.y'].str[1:-1].astype(float)

# extract Q from music filename.
df_Q['music.id_project'] = df_Q.apply(lambda row: getQ(row), axis=1)

# Get Q from mouse click position.
df_Q['Quadrants.participant'] = df_Q.apply(lambda row: categorise(row), axis=1)

# select each music and look for new id
df_log_M['musics']

QperPart=[]

for index, row in df_log_M.iterrows():
    #print(index, row['musics'])
    if row['musics'] == row['musics']:
        toks=row['musics'].split("\\")
        musictok=toks[2]
        # print(musictok, toks[1], index)
        
        #find music in classification dataframe
        musics_dfQ = df_Q.loc[df_Q['music.id'].str.contains(musictok, case=False)]
        
        (musics_dfQ['Quadrants.participant'].values[0])
        QperPart.append(musics_dfQ['Quadrants.participant'].values[0])
    else:
        QperPart.append(np.NaN)
        

# print(QperPart)
df_log_M['musicsQperPart'] = QperPart

df_log_M.head(10)

Participant (idx), run (idx):  04 01


,MusicsSelect,musics,Baseline1.started,Noise.started,Baseline2.started,MusicExcerpt.started,MusicExcerpt.stopped,musicsQperPart
0,MusicListQ2.csv,music\Q2\MT0005202791.wav,27.606404,39.559165,51.559607,63.579621,75.092716,Q2
1,MusicListQ2.csv,music\Q2\MT0011369407.wav,NaN,NaN,NaN,75.579687,87.091594,Q1
2,MusicListQ2.csv,music\Q2\MT0005409948.wav,NaN,NaN,NaN,87.613406,99.121107,Q2
3,MusicListQ2.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MusicListQ4.csv,music\Q4\MT0005129157.wav,99.580519,111.562688,123.563046,135.630240,147.141867,Q1
5,MusicListQ4.csv,music\Q4\MT0013095380.wav,NaN,NaN,NaN,147.630601,159.142138,Q3
6,MusicListQ4.csv,music\Q4\MT0005550441.wav,NaN,NaN,NaN,159.647765,171.151424,Q4
7,MusicListQ4.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MusicListQ3.csv,music\Q3\MT0004131058.wav,171.576276,183.565299,195.571719,207.649537,219.161835,Q1
9,MusicListQ3.csv,music\Q3\MT0010804974.wav,NaN,NaN,NaN,219.599949,231.111449,Q4


In [666]:
# Select subset of columns
df_last_bl=df_log.loc[:,["Baseline3.started", "Baseline3.stopped"]]

# Remove rows with NaN - nor necessary for this event files.
df_last_bl=df_last_bl.dropna(how="any")


In [667]:
# Create pandas dataframe with columns required by the BIDS standard.
bids={'onset':[],'duration':[],'trial_type':[]}
df_bids=pd.DataFrame(bids)

In [668]:
#number of decimals when rounding numbers
decimals=2

# offset of the experiment in seconds (seconds until first baseline.)
exp_offset=df_log_M.iloc[0]["Baseline1.started"]

# considering the log file, add rows for each event.

# For each row:
for i in range(len(df_log_M)-1):
    if not np.isnan(df_log_M.iloc[i]["Baseline1.started"]):
        # Add Baseline trial.
        df_bids.loc[len(df_bids.index)] = [np.round(df_log_M.iloc[i]["Baseline1.started"]-exp_offset, decimals),
                                           np.round(df_log_M.iloc[i]["Noise.started"]-df_log_M.iloc[i]["Baseline1.started"],
                                                    decimals),
                                          'Baseline'] 
        # Add Noise trial.
        df_bids.loc[len(df_bids.index)] = [np.round(df_log_M.iloc[i]["Noise.started"]-exp_offset, decimals),
                                           np.round(df_log_M.iloc[i]["Baseline2.started"]-df_log_M.iloc[i]["Noise.started"],
                                                    decimals),
                                          'Noise'] 
        # Add Baseline2 trial.
        df_bids.loc[len(df_bids.index)] = [np.round(df_log_M.iloc[i]["Baseline2.started"]-exp_offset, decimals),
                                           np.round(df_log_M.iloc[i]["MusicExcerpt.started"]-df_log_M.iloc[i]["Baseline2.started"],
                                                    decimals),
                                          'Baseline'] 

        # Add Music trial.
        df_bids.loc[len(df_bids.index)] = [np.round(df_log_M.iloc[i]["MusicExcerpt.started"]-exp_offset, decimals),
                                           np.round(df_log_M.iloc[i]["MusicExcerpt.stopped"]-df_log_M.iloc[i]["MusicExcerpt.started"], decimals),
                                           df_log_M.iloc[i]["musicsQperPart"]] 
    
    elif not np.isnan(df_log_M.iloc[i]["MusicExcerpt.started"]):
    
        # Add Music trial.
        df_bids.loc[len(df_bids.index)] = [np.round(df_log_M.iloc[i]["MusicExcerpt.started"]-exp_offset, decimals),
                                           np.round(df_log_M.iloc[i]["MusicExcerpt.stopped"]-df_log_M.iloc[i]["MusicExcerpt.started"], decimals),
                                           df_log_M.iloc[i]["musicsQperPart"]] 
    

In [669]:
# Last block.

df_bids.loc[len(df_bids.index)] = [np.round(df_last_bl.iloc[0]["Baseline3.started"]-exp_offset, decimals),
                                       np.round(df_last_bl.iloc[0]["Baseline3.stopped"]-df_last_bl.iloc[0]["Baseline3.started"], decimals),
                                       'Baseline'] 

In [670]:
df_bids

,onset,duration,trial_type
0,0.00,11.95,Baseline
1,11.95,12.00,Noise
2,23.95,12.02,Baseline
3,35.97,11.51,Q2
4,47.97,11.51,Q1
5,60.01,11.51,Q2
6,71.97,11.98,Baseline
7,83.96,12.00,Noise
8,95.96,12.07,Baseline
9,108.02,11.51,Q1


## Create final dataframe

In [671]:
df_bids

,onset,duration,trial_type
0,0.00,11.95,Baseline
1,11.95,12.00,Noise
2,23.95,12.02,Baseline
3,35.97,11.51,Q2
4,47.97,11.51,Q1
5,60.01,11.51,Q2
6,71.97,11.98,Baseline
7,83.96,12.00,Noise
8,95.96,12.07,Baseline
9,108.02,11.51,Q1


In [672]:
# Create tsv file.
fn=f"{sub_id}_ses-{sess_id}_task-{task_id}_{run_id}_events.tsv"
df_bids.to_csv(os.path.join('bids-events-files',fn), sep="\t", index=False)

In [673]:
del df_bids